# Multi-Label Prediction

사용자 정의 클래스를 추가하여, 다중 레이블에 대해서도 쉽게 훈련을 수행할 수 있습니다. 본 핸즈온을 통해 다중 레이블 훈련의 예시를 살펴 보겠습니다.

## 1. MultilabelPredictor Class

사용자 지정 MultilabelPredictor 클래스를 정의하여 각 레이블에 대해 하나씩 TabularPredictor 개체 컬렉션을 관리합니다.

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.core.utils.utils import setup_outputdir
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path

class MultilabelPredictor():
    """ Tabular Predictor for predicting multiple columns in table.
        Creates multiple TabularPredictor objects which you can also use individually.
        You can access the TabularPredictor for a particular label via: `multilabel_predictor.get_predictor(label_i)`

        Parameters
        ----------
        labels : List[str]
            The ith element of this list is the column (i.e. `label`) predicted by the ith TabularPredictor stored in this object.
        path : str
            Path to directory where models and intermediate outputs should be saved.
            If unspecified, a time-stamped folder called "AutogluonModels/ag-[TIMESTAMP]" will be created in the working directory to store all models.
            Note: To call `fit()` twice and save all results of each fit, you must specify different `path` locations or don't specify `path` at all.
            Otherwise files from first `fit()` will be overwritten by second `fit()`.
            Caution: when predicting many labels, this directory may grow large as it needs to store many TabularPredictors.
        problem_types : List[str]
            The ith element is the `problem_type` for the ith TabularPredictor stored in this object.
        eval_metrics : List[str]
            The ith element is the `eval_metric` for the ith TabularPredictor stored in this object.
        consider_labels_correlation : bool
            Whether the predictions of multiple labels should account for label correlations or predict each label independently of the others.
            If True, the ordering of `labels` may affect resulting accuracy as each label is predicted conditional on the previous labels appearing earlier in this list (i.e. in an auto-regressive fashion).
            Set to False if during inference you may want to individually use just the ith TabularPredictor without predicting all the other labels.
        kwargs :
            Arguments passed into the initialization of each TabularPredictor.

    """

    multi_predictor_file = 'multilabel_predictor.pkl'

    def __init__(self, labels, path, problem_types=None, eval_metrics=None, consider_labels_correlation=True, **kwargs):
        if len(labels) < 2:
            raise ValueError("MultilabelPredictor is only intended for predicting MULTIPLE labels (columns), use TabularPredictor for predicting one label (column).")
        self.path = setup_outputdir(path, warn_if_exist=False)
        self.labels = labels
        self.consider_labels_correlation = consider_labels_correlation
        self.predictors = {}  # key = label, value = TabularPredictor or str path to the TabularPredictor for this label
        if eval_metrics is None:
            self.eval_metrics = {}
        else:
            self.eval_metrics = {labels[i] : eval_metrics[i] for i in range(len(labels))}
        problem_type = None
        eval_metric = None
        for i in range(len(labels)):
            label = labels[i]
            path_i = self.path + "Predictor_" + label
            if problem_types is not None:
                problem_type = problem_types[i]
            if eval_metrics is not None:
                eval_metric = self.eval_metrics[i]
            self.predictors[label] = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric, path=path_i, **kwargs)

    def fit(self, train_data, tuning_data=None, **kwargs):
        """ Fits a separate TabularPredictor to predict each of the labels.

            Parameters
            ----------
            train_data, tuning_data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                See documentation for `TabularPredictor.fit()`.
            kwargs :
                Arguments passed into the `fit()` call for each TabularPredictor.
        """
        if isinstance(train_data, str):
            train_data = TabularDataset(train_data)
        if tuning_data is not None and isinstance(tuning_data, str):
            tuning_data = TabularDataset(tuning_data)
        train_data_og = train_data.copy()
        if tuning_data is not None:
            tuning_data_og = tuning_data.copy()
        save_metrics = len(self.eval_metrics) == 0
        for i in range(len(self.labels)):
            label = self.labels[i]
            predictor = self.get_predictor(label)
            if not self.consider_labels_correlation:
                labels_to_drop = [l for l in self.labels if l!=label]
            else:
                labels_to_drop = [labels[j] for j in range(i+1,len(self.labels))]
            train_data = train_data_og.drop(labels_to_drop, axis=1)
            if tuning_data is not None:
                tuning_data = tuning_data_og.drop(labels_to_drop, axis=1)
            print(f"Fitting TabularPredictor for label: {label} ...")
            predictor.fit(train_data=train_data, tuning_data=tuning_data, **kwargs)
            self.predictors[label] = predictor.path
            if save_metrics:
                self.eval_metrics[label] = predictor.eval_metric
        self.save()

    def predict(self, data, **kwargs):
        """ Returns DataFrame with label columns containing predictions for each label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. If label columns are present in this data, they will be ignored. See documentation for `TabularPredictor.predict()`.
            kwargs :
                Arguments passed into the predict() call for each TabularPredictor.
        """
        return self._predict(data, as_proba=False, **kwargs)

    def predict_proba(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `predict_proba()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. See documentation for `TabularPredictor.predict()` and `TabularPredictor.predict_proba()`.
            kwargs :
                Arguments passed into the `predict_proba()` call for each TabularPredictor (also passed into a `predict()` call).
        """
        return self._predict(data, as_proba=True, **kwargs)

    def evaluate(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `evaluate()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to evalate predictions of all labels for, must contain all labels as columns. See documentation for `TabularPredictor.evaluate()`.
            kwargs :
                Arguments passed into the `evaluate()` call for each TabularPredictor (also passed into the `predict()` call).
        """
        data = self._get_data(data)
        eval_dict = {}
        for label in self.labels:
            print(f"Evaluating TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            eval_dict[label] = predictor.evaluate(data, **kwargs)
            if self.consider_labels_correlation:
                data[label] = predictor.predict(data, **kwargs)
        return eval_dict

    def save(self):
        """ Save MultilabelPredictor to disk. """
        for label in self.labels:
            if not isinstance(self.predictors[label], str):
                self.predictors[label] = self.predictors[label].path
        save_pkl.save(path=self.path+self.multi_predictor_file, object=self)
        print(f"MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('{self.path}')")

    @classmethod
    def load(cls, path):
        """ Load MultilabelPredictor from disk `path` previously specified when creating this MultilabelPredictor. """
        path = os.path.expanduser(path)
        if path[-1] != os.path.sep:
            path = path + os.path.sep
        return load_pkl.load(path=path+cls.multi_predictor_file)

    def get_predictor(self, label):
        """ Returns TabularPredictor which is used to predict this label. """
        predictor = self.predictors[label]
        if isinstance(predictor, str):
            return TabularPredictor.load(path=predictor)
        return predictor

    def _get_data(self, data):
        if isinstance(data, str):
            return TabularDataset(data)
        return data.copy()

    def _predict(self, data, as_proba=False, **kwargs):
        data = self._get_data(data)
        if as_proba:
            predproba_dict = {}
        for label in self.labels:
            print(f"Predicting with TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            if as_proba:
                predproba_dict[label] = predictor.predict_proba(data, as_multiclass=True, **kwargs)
            data[label] = predictor.predict(data, **kwargs)
        if not as_proba:
            return data[self.labels]
        else:
            return predproba_dict

## 2. Data preparation and Training

`01_binary_classification.ipynb`와 동일한 데이터셋을 사용합니다.

In [2]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


이번에는 교육 기간, 학력, 개인 소득을 동시에 예측해 보겠습니다.

In [3]:
labels = ['education-num','education','class']  # which columns to predict based on the others
problem_types = ['regression','multiclass','binary']  # type of each prediction problem
save_path = 'ag-02-multilabel'
time_limit = 60

In [4]:
!rm -rf $save_path

사용자 정의 클래스로 훈련을 수행합니다. 만약 다중 레이블의 상관 관계를 고려해야 한다면, `consider_labels_correlation=True`로 설정하고 개별적으로 예측하려면 `consider_labels_correlation=False`로 설정하세요.

In [5]:
multi_predictor = MultilabelPredictor(labels=labels, problem_types=problem_types, path=save_path)
multi_predictor.fit(train_data, time_limit=time_limit)

Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "ag-02-multilabel/Predictor_education-num/"
AutoGluon Version:  0.1.0
Train Data Rows:    500
Train Data Columns: 12
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
NumExpr defaulting to 8 threads.
	Available Memory:                    59691.22 MB
	Train Data (Original)  Memory Usage: 0.26 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...


Fitting TabularPredictor for label: education-num ...


			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('int', [])    : 5 | ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
		('object', []) : 7 | ['workclass', 'marital-status', 'occupation', 'relationship', 'race', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('category', []) : 7 | ['workclass', 'marital-status', 'occupation', 'relationship', 'race', ...]
		('int', [])      : 5 | ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
	0.1s = Fit runtime
	12 features in original data used to generate 12 features in processed data.
	Train Data (Processed) Memory Usage: 0.02 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.15s ...
AutoGluon will gauge predictive performance using evaluation metric: 'root_mean_squared_error'
	To change this, specify the eval_metric arg

	-7.6062	 = Validation root_mean_squared_error score
	37.71s	 = Training runtime
	0.31s	 = Validation runtime
Fitting model: LightGBMLarge ... Training model for up to 5.6s of the 5.6s of remaining time.
	-2.3296	 = Validation root_mean_squared_error score
	0.47s	 = Training runtime
	0.02s	 = Validation runtime


Fitting model: WeightedEnsemble_L2 ... Training model for up to 59.85s of the 3.8s of remaining time.
	-2.1322	 = Validation root_mean_squared_error score
	0.72s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 56.95s ...
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("ag-02-multilabel/Predictor_education-num/")
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "ag-02-multilabel/Predictor_education/"
AutoGluon Version:  0.1.0
Train Data Rows:    500
Train Data Columns: 13
Preprocessing data ...
Fraction of data from classes with at least 10 examples that will be kept for training models: 0.976
Train Data Class Count: 11
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    56929.19 MB
	Train Data (Original)  Memory Usage: 0.25 MB (0.0% of available memory)
	Inferring data type of each feature based on column val

...

Fitting TabularPredictor for label: class ...


	0.84	 = Validation accuracy score
	0.87s	 = Training runtime
	0.11s	 = Validation runtime
Fitting model: RandomForestEntr ... Training model for up to 58.88s of the 58.88s of remaining time.
	0.83	 = Validation accuracy score
	0.86s	 = Training runtime
	0.11s	 = Validation runtime
Fitting model: ExtraTreesGini ... Training model for up to 57.89s of the 57.89s of remaining time.
	0.83	 = Validation accuracy score
	0.76s	 = Training runtime
	0.11s	 = Validation runtime
Fitting model: ExtraTreesEntr ... Training model for up to 57.0s of the 56.99s of remaining time.
	0.84	 = Validation accuracy score
	0.76s	 = Training runtime
	0.11s	 = Validation runtime
Fitting model: KNeighborsUnif ... Training model for up to 56.1s of the 56.1s of remaining time.
	0.73	 = Validation accuracy score
	0.0s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: KNeighborsDist ... Training model for up to 55.99s of the 55.99s of remaining time.
	0.65	 = Validation accuracy score
	0.0s	 = Training 

	0.83	 = Validation accuracy score
	18.05s	 = Training runtime
	0.26s	 = Validation runtime
Fitting model: LightGBMLarge ... Training model for up to 31.09s of the 31.09s of remaining time.
	0.83	 = Validation accuracy score
	0.5s	 = Training runtime
	0.02s	 = Validation runtime


Fitting model: WeightedEnsemble_L2 ... Training model for up to 59.87s of the 29.25s of remaining time.
	0.86	 = Validation accuracy score
	0.54s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 31.32s ...
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("ag-02-multilabel/Predictor_class/")


MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('ag-02-multilabel/')


## 3. Inference and Evaluation

In [6]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
test_data = test_data.sample(n=subsample_size, random_state=0)
test_data_nolab = test_data.drop(columns=labels)  # unnecessary, just to demonstrate we're not cheating here
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
5454,41,Self-emp-not-inc,408498,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States
6111,39,Private,746786,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,55,United-States
5282,50,Private,62593,Married-civ-spouse,Farming-fishing,Husband,Asian-Pac-Islander,Male,0,0,40,United-States
3046,31,Private,248178,Married-civ-spouse,Other-service,Husband,Black,Male,0,0,35,United-States
2162,43,State-gov,52849,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States


In [7]:
multi_predictor = MultilabelPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained multilabel predictor from file

predictions = multi_predictor.predict(test_data_nolab)
print("Predictions:  \n", predictions)

Predicting with TabularPredictor for label: education-num ...
Predicting with TabularPredictor for label: education ...
Predicting with TabularPredictor for label: class ...
Predictions:  
       education-num      education   class
5454      10.618733   Some-college    >50K
6111      12.840778        HS-grad    >50K
5282       9.452595        HS-grad    >50K
3046       9.474932        HS-grad   <=50K
2162      12.477719        HS-grad    >50K
...             ...            ...     ...
6965       9.629266        HS-grad   <=50K
4762       8.992409           11th   <=50K
234       10.347326   Some-college   <=50K
6291      10.311174   Some-college   <=50K
9575      10.015663   Some-college   <=50K

[500 rows x 3 columns]


In [8]:
evaluations = multi_predictor.evaluate(test_data)
print(evaluations)
print("Evaluated using metrics:", multi_predictor.eval_metrics)

Evaluating TabularPredictor for label: education-num ...
Predictive performance on given data: root_mean_squared_error = 2.185123915690999
Evaluating TabularPredictor for label: education ...
Predictive performance on given data: accuracy = 0.216
Evaluating TabularPredictor for label: class ...
Predictive performance on given data: accuracy = 0.818
{'education-num': 2.185123915690999, 'education': 0.216, 'class': 0.818}
Evaluated using metrics: {'education-num': root_mean_squared_error, 'education': accuracy, 'class': accuracy}


다음과 같이 레이블 중 하나에 대해 TabularPredictor 단일 레이블을 예측할 수도 있습니다.

In [9]:
predictor_class = multi_predictor.get_predictor('class')
predictor_class.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.86,0.015572,1.450373,0.015572,1.450373,1,True,9
1,WeightedEnsemble_L2,0.86,0.016405,1.990871,0.000834,0.540498,2,True,14
2,XGBoost,0.85,0.013783,0.518251,0.013783,0.518251,1,True,10
3,LightGBM,0.85,0.018446,0.314851,0.018446,0.314851,1,True,7
4,ExtraTreesEntr,0.84,0.111939,0.756836,0.111939,0.756836,1,True,4
5,RandomForestGini,0.84,0.112039,0.866010,0.112039,0.866010,1,True,1
6,LightGBMLarge,0.83,0.017709,0.504494,0.017709,0.504494,1,True,13
7,LightGBMXT,0.83,0.017916,0.250356,0.017916,0.250356,1,True,8
8,RandomForestEntr,0.83,0.111526,0.857153,0.111526,0.857153,1,True,2
9,ExtraTreesGini,0.83,0.111748,0.757162,0.111748,0.757162,1,True,3
